## A neural network to solve the DC-OPF

### Data processing

In [1]:
# Importing the libraries
import torch
from torchviz import make_dot
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# from collections import defaultdict
# split a univariate sequence into samples
def split_sequenceUStep(sequence, n_steps_in):
    X, y = list(), list()
    # X, y = defaultdict(list), defaultdict(list)
    
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [3]:
# Define the study case
CaseName = '3-bus'

In [4]:
# Load the data
df_dict_gen   = pd.read_csv('01.Data/'+ CaseName+'/ANN_Dict_Generation_'        +CaseName+'.csv', header=0, index_col=[0])
df_data_gen   = pd.read_csv('01.Data/'+ CaseName+'/ANN_Data_Generation_'        +CaseName+'.csv', header=0, index_col=[0])
df_demand     = pd.read_csv('01.Data/'+ CaseName+'/ANN_Data_Demand_'            +CaseName+'.csv', header=0, index_col=[0,1,2])
df_generation = pd.read_csv('01.Data/'+ CaseName+'/ANN_Result_GenerationEnergy_'+CaseName+'.csv', header=0, index_col=[0,1,2])

In [5]:
df_data_gen['MinimumPower']['CCGT_1']

0.0

In [6]:
df_demand     = df_demand.stack().reset_index().pivot_table(index=['level_2'], columns=['level_3'], values=0, aggfunc='sum')
df_generation = df_generation.reset_index().pivot_table(index=['LoadLevel'], columns='Unit', values='GWh', aggfunc='sum')

In [7]:
df_data = pd.concat([df_demand, df_generation], axis=1)

In [8]:
# normalize the dataset
# df_data = (df_data - df_data.min()) / (df_data.max() - df_data.min())
df_data['Node_1'] /= 1000
df_data['Node_2'] /= 1000
df_data['Node_3'] /= 1000

In [9]:
# from GWh to MWh
df_data *= 1000

In [10]:
# normalize the demand dataset
df_data['Node_1'] = (df_data['Node_1'] - df_data['Node_1'].min()) / (df_data['Node_1'].max() - df_data['Node_1'].min())
df_data['Node_2'] = (df_data['Node_2'] - df_data['Node_2'].min()) / (df_data['Node_2'].max() - df_data['Node_2'].min())
df_data['Node_3'] = (df_data['Node_3'] - df_data['Node_3'].min()) / (df_data['Node_3'].max() - df_data['Node_3'].min())

In [11]:
# normalize the generation dataset
df_data['CCGT_1'] = (df_data['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) / (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1'])
df_data['CCGT_2'] = (df_data['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) / (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2'])

In [12]:
# split into input (demand) and outputs (generation)
X = df_data.iloc[:,:3].values
y = df_data.iloc[:,3:6].values


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [14]:
# Convert the data into PyTorch tensors
train_inputs  = torch.from_numpy(X_train)
train_targets = torch.from_numpy(y_train)
test_inputs   = torch.from_numpy(X_test)
test_targets  = torch.from_numpy(y_test)

In [15]:
# Define the neural network model
class aNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(input_size, hidden_size1)
        torch.nn.init.kaiming_uniform_(self.hidden_layer1.weight, a=0)
        self.hidden_layer2 = torch.nn.Linear(hidden_size1, hidden_size2)
        self.hidden_layer3 = torch.nn.Linear(hidden_size2, hidden_size3)
        self.output_layer = torch.nn.Linear(hidden_size3, output_size)

        # define the device to use (GPU or CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def forward(self, input):
        hidden1 = torch.relu(self.hidden_layer1(input))
        hidden2 = torch.relu(self.hidden_layer2(hidden1))
        # hidden3 = torch.relu(self.hidden_layer3(hidden2))
        hidden3 = self.hidden_layer3(hidden2)
        output = self.output_layer(hidden3)
        return output

In [16]:
# Initialize the model and optimizer
model = aNN(input_size=3, hidden_size1=32, hidden_size2=16, hidden_size3=8, output_size=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Train the model
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    train_predictions = model(train_inputs.float())
    train_loss = torch.nn.MSELoss()(train_predictions.float().squeeze(), train_targets.float())

    # Backward pass
    # optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Print the training loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

Epoch 10, Train Loss: 0.5714046955108643
Epoch 20, Train Loss: 0.4193643033504486
Epoch 30, Train Loss: 0.2795654237270355
Epoch 40, Train Loss: 0.15618497133255005
Epoch 50, Train Loss: 0.06562673300504684
Epoch 60, Train Loss: 0.020392518490552902
Epoch 70, Train Loss: 0.013558031059801579
Epoch 80, Train Loss: 0.013473832048475742
Epoch 90, Train Loss: 0.010369607247412205
Epoch 100, Train Loss: 0.008646944537758827
Epoch 110, Train Loss: 0.007585547398775816
Epoch 120, Train Loss: 0.0065331063233315945
Epoch 130, Train Loss: 0.005678690504282713
Epoch 140, Train Loss: 0.004988003056496382
Epoch 150, Train Loss: 0.004419767763465643
Epoch 160, Train Loss: 0.003971650265157223
Epoch 170, Train Loss: 0.003623749129474163
Epoch 180, Train Loss: 0.003354820189997554
Epoch 190, Train Loss: 0.003144751535728574
Epoch 200, Train Loss: 0.00297424104064703
Epoch 210, Train Loss: 0.0028308280743658543
Epoch 220, Train Loss: 0.0027059337589889765
Epoch 230, Train Loss: 0.002594149438664317
Epo

In [18]:
# Evaluate the model on the test set
test_predictions = model(test_inputs.float())
test_loss = torch.nn.MSELoss()(test_predictions.float().squeeze(), test_targets.float())
print(f'Test Loss: {test_loss.item()}')

Test Loss: 4.4005199015373364e-05


In [19]:
test_predictions.float()

tensor([[ 0.0132,  0.6575],
        [ 0.0061,  0.9973],
        [ 0.1690,  0.9952],
        ...,
        [-0.0029,  0.9301],
        [-0.0031,  0.8616],
        [ 0.0910,  1.0022]], grad_fn=<AddmmBackward0>)

In [20]:
# Save the trained model
torch.save(model.state_dict(), "model.pth")

In [21]:
# Convert the test predictions and test output to NumPy arrays
test_targets = test_targets.detach().numpy()
test_predictions = test_predictions.detach().numpy()

In [22]:
test_targets_df = pd.DataFrame(test_targets, columns=['G1_target','G2_target'])
test_targets_df['G1_target'] = test_targets_df['G1_target'] * (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) + df_data_gen['MinimumPower']['CCGT_1']
test_targets_df['G2_target'] = test_targets_df['G2_target'] * (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) + df_data_gen['MinimumPower']['CCGT_2']
test_targets_df.to_csv('test_targets.csv', index=False)
test_predictions_df = pd.DataFrame(test_predictions, columns=['G1_estimate','G2_estimate'])
test_predictions_df['G1_estimate'] = test_predictions_df['G1_estimate'] * (df_data_gen['MaximumPower']['CCGT_1'] - df_data_gen['MinimumPower']['CCGT_1']) + df_data_gen['MinimumPower']['CCGT_1']
test_predictions_df['G2_estimate'] = test_predictions_df['G2_estimate'] * (df_data_gen['MaximumPower']['CCGT_2'] - df_data_gen['MinimumPower']['CCGT_2']) + df_data_gen['MinimumPower']['CCGT_2']
test_predictions_df.to_csv('test_predictions.csv', index=False)

In [23]:
time_df = pd.DataFrame({'LoadLevel': pd.date_range(start='2023-05-04 00:00:00', periods=len(test_predictions_df), freq='H')})
frames = [time_df, test_predictions_df, test_targets_df]
result = pd.concat(frames, axis=1).set_index('LoadLevel')

In [24]:
source = result.stack().reset_index().rename(columns={'level_1':'Demand', 0:'Value'})

In [25]:
import altair as alt

lines = (
    alt.Chart(source)
    .mark_line()
    .encode(x="LoadLevel", y="Value", color="Demand")
).properties(width=1500, height=500)
lines.save('Plot.html', embed_options={'renderer':'svg'})